# Multiclass Classification Model Artifact Upload

This notebook will go over how we can upload a multiclass classification model artifact to Fiddler and unlock the full power of explainability for your model.

We will be using a model trained on a modified version of the classic [iris dataset](https://archive.ics.uci.edu/ml/datasets/Iris).

---

The process is similar to registering a model without uploading an artifact. Just follow these steps:
1. Connect to Fiddler
2. Upload a baseline dataset
3. Create a model package
4. Upload the model package
5. Test model predictions

## Choose your framework

Fiddler supports multiple frameworks.

Choose the one you want and **run the code block** to see a customized integration tutorial.

In [ ]:
#@title  { vertical-output: true, display-mode: "form" }
FRAMEWORK = "XGBoost" #@param ["scikit-learn", "TensorFlow (SavedModel)", "TensorFlow (HDF5)", "XGBoost"]
if FRAMEWORK == 'scikit-learn':
    MODEL_PACKAGE_URL = 'https://github.com/fiddler-labs/fiddler-samples/blob/new-examples/content_root/tutorial/model_artifact_upload/multiclass_classification/scikit-learn/model_package.zip?raw=true'
elif FRAMEWORK == 'TensorFlow (SavedModel)':
    MODEL_PACKAGE_URL = 'https://github.com/fiddler-labs/fiddler-samples/blob/new-examples/content_root/tutorial/model_artifact_upload/multiclass_classification/tensorflow-savedmodel/model_package.zip?raw=true'
elif FRAMEWORK == 'TensorFlow (HDF5)':
    MODEL_PACKAGE_URL = 'https://github.com/fiddler-labs/fiddler-samples/blob/new-examples/content_root/tutorial/model_artifact_upload/multiclass_classification/tensorflow-hdf5/model_package.zip?raw=true'
elif FRAMEWORK == 'XGBoost':
    MODEL_PACKAGE_URL = 'https://github.com/fiddler-labs/fiddler-samples/blob/new-examples/content_root/tutorial/model_artifact_upload/multiclass_classification/xgboost/model_package.zip?raw=true'

DATA_FILE_URL = 'https://raw.githubusercontent.com/fiddler-labs/fiddler-samples/new-examples/content_root/tutorial/model_artifact_upload/multiclass_classification/iris_data.csv'

from urllib.request import urlretrieve
from zipfile import ZipFile

print("Downloading model files...")

data_file_handle, _ = urlretrieve(DATA_FILE_URL, 'iris_data.csv')

!rm -rf model_package;
!mkdir model_package;
model_package_file_handle, _ = urlretrieve(MODEL_PACKAGE_URL)
zip_file = ZipFile(model_package_file_handle, 'r')
zip_file.extractall('model_package')
!rm $model_package_file_handle;

print("Model files downloaded.")

## 0. Imports

In [ ]:
!pip install -q fiddler-client;

import numpy as np
import pandas as pd
import yaml
import pickle
import pathlib

import fiddler as fdl

print(f"Running client version {fdl.__version__}")

## 1. Connect to Fiddler

Connect to the Fiddler client as you normally would.

For more information on how this can be done, see our [quick start guide](https://docs.fiddler.ai/pages/getting-started/quick-start/).

In [ ]:
URL = #
ORG_ID = #
AUTH_TOKEN = #


client = fdl.FiddlerApi(
    url=URL,
    org_id=ORG_ID,
    auth_token=AUTH_TOKEN
)

Next, create a new project for our regression model.

In [ ]:
PROJECT_ID = 'multiclass_example'

client.create_project(PROJECT_ID)

## 2. Upload a baseline dataset

Now we need to upload a dataset to use as a baseline for monitoring and explainability.

*For more information on how to design a baseline dataset, [click here](#).*

---

**We've already loaded in the CSV file for you.** Just run the following code block

In [ ]:
PATH_TO_BASELINE_CSV = 'iris_data.csv'

baseline_df = pd.read_csv(PATH_TO_BASELINE_CSV)
baseline_df

You can use this DataFrame to construct a Fiddler `DatasetInfo` object.

In [ ]:
dataset_info = fdl.DatasetInfo.from_dataframe(baseline_df)
dataset_info

Then, upload the dataset to Fiddler.

In [ ]:
DATASET_ID = 'iris_data'

client.upload_dataset(
    project_id=PROJECT_ID,
    dataset_id=DATASET_ID,
    dataset={
        'baseline': baseline_df
    },
    info=dataset_info
)

## 3. Create a model package

In order for Fiddler to use your model for explainability, you'll need to create a **model package**. This is a collection of files that helps Fiddler run your model.

---

Your model package should include:
1. Your model artifact
2. A YAML file containing specifications for your model
3. Any other serialized objects needed to run your model (such as data transformers)
4. A `package.py` script (more on this later)

Start by specifying a unique model ID.

In [ ]:
MODEL_ID = 'iris_classifier'

The next step is to **create a folder and store your model artifact there**.

We've already loaded in a sample model artifact for you. We've stored it in the `model_package` directory.

---

Next, let's create a `ModelInfo` object that can be used to generate the YAML file.

In [ ]:
# Specify column types
target = 'species'
outputs = [
    'probability_0',
    'probability_1',
    'probability_2'
]
features = [
    'sepal_length',
    'sepal_width',
    'petal_length',
    'petal_width'
]
    
# Generate ModelInfo
model_info = fdl.ModelInfo.from_dataset_info(
    dataset_info=dataset_info,
    dataset_id=DATASET_ID,
    input_type=fdl.ModelInputType.TABULAR,
    model_task=fdl.ModelTask.MULTICLASS_CLASSIFICATION,
    target=target,
    outputs=outputs,
    features=features,
    categorical_target_class_details=[0, 1, 2]
)
model_info

Now, let's convert the object to YAML format and store it with the artifact file.

In [ ]:
with open('model_package/model.yaml', 'w') as yaml_file:
    yaml.dump({'model': model_info.to_dict()}, yaml_file)

The final item required for the model package is a `package.py` script. This is what tells Fiddler how to interface with a particular model.

We've loaded in a sample `package.py`. Run the following code block to see what it looks like.

In [ ]:
with open('model_package/package.py', 'r') as py_file:
    print(py_file.read())

***Note:*** This `package.py` script has been customized specifically for this example model. To learn more about customizing `package.py` for your own model, [click here](#).

## 4. Upload the model package

Almost done! The last step is to upload the model package you've created.

Just run the following code block.

In [ ]:
client.upload_model_package(
    artifact_path=pathlib.Path('model_package'),
    project_id=PROJECT_ID,
    model_id=MODEL_ID
)

## 5. Test model predictions

We can run predictions using Fiddler's backend to **test the model and make sure everything works properly**.

Just run the following code block.

In [ ]:
client.run_model(
    project_id=PROJECT_ID,
    model_id=MODEL_ID,
    df=baseline_df
)

## All done!

---


**Questions?**  
  
Check out [our docs](https://docs.fiddler.ai/) for a more detailed explanation of what Fiddler has to offer.

If you're still looking for answers, fill out a ticket on [our support page](https://fiddlerlabs.zendesk.com/) and we'll get back to you shortly.